In [1]:

import sys
import numba
import pandas as pd
import numpy as np
from numpy import array
import math
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba
import ta
from scipy import stats
from processing_constants import ALL_CONTINUOUS_COLUMNS, HIGH_NAN_COLUMNS, WHOLE_MARKET_COLUMNS
from processing_constants import BOLLINGER_PREDICTION_COLUMNS, BOLLINGER_VALUE_COLUMNS
from optimise_dataframe import optimise_df
import glob
from sklearn import preprocessing
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, HistGradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib
import matplotlib.pyplot as plt



/Users/neilkloot/anaconda/lib/python3.6/site-packages/numba/errors.py:131: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
train_data = pd.read_pickle(
    'data/ml-ta-discretised-training-data-201908262001.pkl.gz', compression='gzip')

print(train_data.dtypes)


week_starting                                 object
lastTradePriceOnly                           float64
adjustedPrice                                float64
volume                                       float64
previousClose                                float64
change                                       float64
changeInPercent                              float64
52WeekHigh                                   float64
52WeekLow                                    float64
changeFrom52WeekHigh                         float64
changeFrom52WeekLow                          float64
percebtChangeFrom52WeekHigh                  float64
percentChangeFrom52WeekLow                   float64
allordpreviousclose                          float64
allordchange                                 float64
allorddayshigh                               float64
allorddayslow                                float64
allordpercebtChangeFrom52WeekHigh            float64
allordpercentChangeFrom52WeekLow             f

In [4]:
train_data.memory_usage().sum()

9620709056

In [5]:
train_data = optimise_df(train_data)

In [6]:
train_data.memory_usage().sum()

4423988310

In [ ]:
# Use full training set (proportion 1)
print('Setting target dfs')
train_target_oh = train_data[bin_names]
train_target_le = train_data['class_index']
print('Setting symbol df')
train_symbols = train_data['symbol']

cols_to_drop = bin_names.copy()
cols_to_drop.extend(['symbol', 'target', 'class', 'class_index'])

print('Dropping cols')
train_data.drop(cols_to_drop, axis=1, inplace=True, errors='ignore')

print('Training data memory', train_data.memory_usage().sum())

In [ ]:
bin_names = ['strong_sell', 'sell', 'hold', 'buy', 'strong_buy']

classifiers = [
    ('Random Forest',
     RandomForestClassifier(
         max_depth=20, n_estimators=505, min_samples_split=2, min_samples_leaf=1)
     ),
    ('Adaboost Random Forest',
     AdaBoostClassifier(RandomForestClassifier(
         max_depth=20, n_estimators=505, min_samples_split=2, min_samples_leaf=1), n_estimators=100)
     ),
    ('Histogram Gradient Boosting',
     HistGradientBoostingClassifier(
         max_iter=343, max_leaf_nodes=623, learning_rate=0.1, min_samples_leaf=2, l2_regularization=0.)
     )
]


print('Training voting classifier with soft voting')
voting_clf = VotingClassifier(
    estimators=classifiers, voting='soft') #, n_jobs=8)
voting_clf = voting_clf.fit(train_data, train_target_le)


Training voting classifier with soft voting


In [ ]:
joblib.dump(voting_clf, './clf-model.joblib.z')



In [ ]:
# Execute backtest
train_x, train_y_le, train_symbols, test_x, test_y_le, test_symbols = split_dataset(
    train_data, 1.)
# del test_x, test_y_le, test_symbols
train_pred_y = model.predict(train_x)
evaluate_classifier(train_y_le, train_pred_y, bin_names)